In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
from sklearn.utils import shuffle
from numpy import asarray

import sys
sys.path.append('../')
from Utilities.model_visualization import model_to_png

print('import done')

C:\Users\Javapocalypse\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


import done


In [11]:
dataset_train = "Dataset1/queensland/train.xlsx"
dataset_test = "Dataset1/queensland/test.xlsx"

In [12]:
train = pd.read_excel(dataset_train)
test = pd.read_excel(dataset_test)

train = train.sample(frac=1)
test = test.sample(frac=1)

print(train.head())
print('-'*30)
print(train['label'].value_counts())
print('-'*30)

# train = train[train['text'].str.len()  15]

# print('x'*50)
# # df[df['column name'].map(len) < 2]
# print(train.head())
# print('-'*30)
# print(train['label'].value_counts())


                tweet_id                                               text  \
1963  297282655832121024                                Ill money months I    
3981  296005918695500992  Flood crisis claims fourth victim A threeyearo...   
5582  296003966221824000  Were still updating Bowls Queensland website i...   
4893  296082334241913984         Deadly flood waters rise eastern Australia   
628   296365043640041024  Hate say time long overdue but time long overd...   

             label  
1963  not_relevant  
3981      relevant  
5582      relevant  
4893      relevant  
628   not_relevant  
------------------------------
relevant        3248
not_relevant    2771
Name: label, dtype: int64
------------------------------


In [13]:
# train['text'][0]
# train['label'][0]
dataColumn = 'text'
labelColumn = 'label'

In [14]:
tags = train[labelColumn]
texts = train[dataColumn]

tags_Y = test[labelColumn]
texts_Y = test[dataColumn]

In [15]:
num_max = 1000
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags.astype(str))
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')
print(tags[:5])
print(mat_texts[:5])
print(tags.shape,mat_texts.shape)


# For testing data
le_Y = LabelEncoder()
tags_Y = le_Y.fit_transform(tags_Y.astype(str))
tok_Y = Tokenizer(num_words=num_max)
tok_Y.fit_on_texts(texts_Y)
mat_texts_Y = tok.texts_to_matrix(texts_Y,mode='count')

[0 1 1 1 0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(6019,) (6019, 1000)


In [16]:
# for cnn preproces
max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
print(cnn_texts_seq[0])
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)
print(cnn_texts_mat[0])
print(cnn_texts_mat.shape)



# For testing data
cnn_texts_seq_Y = tok.texts_to_sequences(texts_Y)
cnn_texts_mat_Y = sequence.pad_sequences(cnn_texts_seq_Y,maxlen=max_len)

[208, 424, 517, 4]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0 208 424 517   4]
(6019, 100)


In [17]:
def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=12,verbose=1,validation_split=0.25)

In [18]:
def get_simple_model():
    model = Sequential()
    model.add(Dense(512, input_shape=(num_max,)))
    model.add(Activation('sigmoid'))
    model.add(Dense(1))
    model.add(Activation('softmax'))
    model.summary()
    model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
    print('compile done')
    return model

In [22]:
m1 = get_simple_model()
model_to_png(m1, 'model_1')
check_model(m1,mat_texts,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_7 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 513       
_________________________________________________________________
activation_8 (Activation)    (None, 1)                 0         
Total params: 513,025
Trainable params: 513,025
Non-trainable params: 0
_________________________________________________________________
compile done
Train on 4514 samples, validate on 1505 samples
Epoch 1/12
4514/4514 [==============================] - 1s 219us/step - loss: 7.2613 - acc: 0.5445 - val_loss: 7.5740 - val_acc: 0.5249
Epoch 2/12
4514/4514 [==============================] - 1s 174us/step -

In [23]:
def get_cnn_model_v1():   
    model = Sequential()
    # Starting off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [24]:
m2 = get_cnn_model_v1()
model_to_png(m2, 'model_2')
check_model(m2,cnn_texts_mat,tags)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 20)           20000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 20)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 64)            3904      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
___________________________________________________________

In [25]:
def get_cnn_model_v2():
    model = Sequential()
    # Increased Output Dim
    model.add(Embedding(1000,
                        50,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [26]:
m3 = get_cnn_model_v2()
model_to_png(m3, 'model_3')
check_model(m3,cnn_texts_mat,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 98, 64)            9664      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_11 (Activation)   (None, 256)               0         
__________

In [27]:
def get_cnn_model_v3():    # added filter
    model = Sequential()

    model.add(Embedding(1000,
                        50,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256, #!!!!!!!!!!!!!!!!!!!
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [28]:
m4 = get_cnn_model_v3()
model_to_png(m4, 'model_4')
check_model(m4,cnn_texts_mat,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 98, 256)           38656     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
activation_13 (Activation)   (None, 128)               0         
__________

In [29]:
def get_cnn_model_v4():    # Pre Trained Embeddings
    
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open('Embeddings/glove.6B.100d.txt', encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    
    # create a weight matrix for words in training docs
    embedding_matrix = np.zeros((len(tok.word_index) + 1, 100))
    for word, i in tok.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    
    model = Sequential()    
    e = Embedding(len(tok.word_index) + 1, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)
    model.add(e)  #!!!!!!!!!!!!!!!!!!!
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [30]:
m5 = get_cnn_model_v4()
model_to_png(m5, 'model_5')
check_model(m5,cnn_texts_mat,tags)

Found 400000 word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          908400    
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 98, 128)           38528     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
activation_15 (Activation)   (None, 128)         

In [31]:
scores = m1.evaluate(mat_texts_Y, tags_Y)
print('{0:.2f}'.format(scores[1]*100) + '% Accurate ANN')
print('-'*50)
scores = m3.evaluate(cnn_texts_mat_Y, tags_Y)
print('{0:.2f}'.format(scores[1]*100) + '% Accurate CNN with Embedding')
print('-'*50)
scores = m4.evaluate(cnn_texts_mat_Y, tags_Y)
print('{0:.2f}'.format(scores[1]*100) + '% Accurate CNN with Embedding and more Filters')
print('-'*50)
scores = m5.evaluate(cnn_texts_mat_Y, tags_Y)
print('{0:.2f}'.format(scores[1]*100) + '% Accurate CNN with PRe Trained GLOVE Embedding')

3011/3011 [==============================] - 0s 58us/step
53.97% Accurate ANN
--------------------------------------------------
3011/3011 [==============================] - 0s 53us/step
93.92% Accurate CNN with Embedding
--------------------------------------------------
3011/3011 [==============================] - 0s 67us/step
94.75% Accurate CNN with Embedding and more Filters
--------------------------------------------------
3011/3011 [==============================] - 0s 64us/step
95.78% Accurate CNN with PRe Trained GLOVE Embedding
